In [1]:
%cd /content/
!git clone https://github.com/pbaylies/stylegan2-ada
%cd /content/stylegan2-ada
!mkdir out

/content
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 361, done.
remote: Total 361 (delta 0), reused 0 (delta 0), pack-reused 361
Receiving objects: 100% (361/361), 61.57 MiB | 29.23 MiB/s, done.
Resolving deltas: 100% (196/196), done.
/content/stylegan2-ada


In [2]:
cd /content/stylegan2-ada

/content/stylegan2-ada


In [3]:
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


In [4]:
!pip install opensimplex

In [9]:
# FFHQ
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

--2022-03-01 15:40:20--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 52.84.159.86, 52.84.159.50, 52.84.159.125, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|52.84.159.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381646055 (364M) [binary/octet-stream]
Saving to: ‘ffhq.pkl.1’

ffhq.pkl.1          100%[===================>] 363.97M   184MB/s    in 2.0s    

2022-03-01 15:40:23 (184 MB/s) - ‘ffhq.pkl.1’ saved [381646055/381646055]



In [7]:
# Metfaces
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl

--2022-03-01 15:33:54--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 52.84.159.125, 52.84.159.86, 52.84.159.50, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|52.84.159.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381646596 (364M) [binary/octet-stream]
Saving to: ‘metfaces.pkl’

metfaces.pkl        100%[===================>] 363.97M  24.1MB/s    in 29s     

2022-03-01 15:34:24 (12.7 MB/s) - ‘metfaces.pkl’ saved [381646596/381646596]



In [4]:
# Import Libs
import tensorflow as tf
import argparse
import sys
import os
import subprocess
import pickle
import re
from projector import Projector
import tqdm
import imageio
import copy

import scipy
import numpy as np
import PIL.Image

import dnnlib
import dnnlib.tflib as tflib

os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
import cv2
import moviepy.editor
from opensimplex import OpenSimplex

import warnings # mostly numpy warnings for me
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [43]:
def create_image_grid(images, grid_size=None):
    '''
    Args:
        images (np.array): images to place on the grid
        grid_size (tuple(int, int)): size of grid (grid_w, grid_h)
    Returns:
        grid (np.array): image grid of size grid_size
    '''
    # Some sanity check:
    assert images.ndim == 3 or images.ndim == 4
    num, img_h, img_w = images.shape[0], images.shape[1], images.shape[2]
    if grid_size is not None:
        grid_w, grid_h = tuple(grid_size)
    else:
        grid_w = max(int(np.ceil(np.sqrt(num))), 1)
        grid_h = max((num - 1) // grid_w + 1, 1)
    # Get the grid
    grid = np.zeros(
        [grid_h * img_h, grid_w * img_w] + list(images.shape[-1:]), dtype=images.dtype
    )
    for idx in range(num):
        x = (idx % grid_w) * img_w
        y = (idx // grid_w) * img_h
        grid[y : y + img_h, x : x + img_w, ...] = images[idx]
    return grid

def render_from_zs(Gs, zs, w_space=False, grid=False, class_idx=None, prefix='', outdir='out'):
    images = []
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    label = np.zeros([1] + Gs.input_shapes[1][1:])
    if class_idx is not None:
        label[:, class_idx] = 1
    for i_z, z in enumerate(zs):
        print('Generating image for latent code (%d/%d) ...' % (i_z, len(zs)))
        if w_space:
            image = Gs.components.synthesis.run(z, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
        else:
            image = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
        images.append(image[0])
        PIL.Image.fromarray(image[0], 'RGB').save(f'{outdir}/z{i_z:04d}.png')

    # If user wants to save a grid of the generated images
    if grid:
        print('Generating image grid...')
        PIL.Image.fromarray(create_image_grid(np.array(images)), 'RGB').save(f'{outdir}/grid{prefix}.png')

In [95]:
# FFHQ laden
tflib.init_tf()
with dnnlib.util.open_url("ffhq.pkl") as fp:
    _, _, ffhq = pickle.load(fp)

In [11]:
# Met laden
tflib.init_tf()
with dnnlib.util.open_url("metfaces.pkl") as fp:
    _, _, met = pickle.load(fp)

In [91]:
def replace(depth=90):
    init = tf.initialize_all_variables()
    session = tf.InteractiveSession()
    session.run(init)

    # FFHQ laden
    tflib.init_tf()
    with dnnlib.util.open_url("metfaces.pkl") as fp:
        _, _, met = pickle.load(fp)

    # get keys
    keys = [key for key in met.vars][-depth:]

    # Met laden
    with dnnlib.util.open_url("ffhq.pkl") as fp:
        _, _, ffhq = pickle.load(fp)

    # replace
    for key in keys:
        session.run(ffhq.vars[key].assign(met.vars[key]))

    #assign_op = layer.assign(met.vars[keys[0]])
    #session.run(assign_op)
    return ffhq

In [97]:
fused = replace(4)

In [96]:
# 12 Zufällige Bilder generieren.
render_from_zs(fused, zs, grid=True)

Generating image for latent code (0/12) ...
Generating image for latent code (1/12) ...
Generating image for latent code (2/12) ...
Generating image for latent code (3/12) ...
Generating image for latent code (4/12) ...
Generating image for latent code (5/12) ...
Generating image for latent code (6/12) ...
Generating image for latent code (7/12) ...
Generating image for latent code (8/12) ...
Generating image for latent code (9/12) ...
Generating image for latent code (10/12) ...
Generating image for latent code (11/12) ...
Generating image grid...


In [20]:
for count, layer in enumerate(fused.list_layers()):
    print(count, layer)

0 ('latents_in', <tf.Tensor 'G_main_4/latents_in:0' shape=(?, 512) dtype=float32>, [])
1 ('labels_in', <tf.Tensor 'G_main_4/labels_in:0' shape=(?, 0) dtype=float32>, [])
2 ('G_mapping/Normalize', <tf.Tensor 'G_main_4/G_mapping/Normalize/mul:0' shape=(?, 512) dtype=float32>, [])
3 ('G_mapping/Dense0', <tf.Tensor 'G_main_4/G_mapping/Dense0/IdentityN:0' shape=(?, 512) dtype=float32>, [<tf.Variable 'G_mapping_4/Dense0/weight:0' shape=(512, 512) dtype=float32_ref>, <tf.Variable 'G_mapping_4/Dense0/bias:0' shape=(512,) dtype=float32_ref>])
4 ('G_mapping/Dense1', <tf.Tensor 'G_main_4/G_mapping/Dense1/IdentityN:0' shape=(?, 512) dtype=float32>, [<tf.Variable 'G_mapping_4/Dense1/weight:0' shape=(512, 512) dtype=float32_ref>, <tf.Variable 'G_mapping_4/Dense1/bias:0' shape=(512,) dtype=float32_ref>])
5 ('G_mapping/Dense2', <tf.Tensor 'G_main_4/G_mapping/Dense2/IdentityN:0' shape=(?, 512) dtype=float32>, [<tf.Variable 'G_mapping_4/Dense2/weight:0' shape=(512, 512) dtype=float32_ref>, <tf.Variable 

In [ ]:
fused.list_layers[33:49] = 